In [27]:
import numpy as np
NORMAL = "normal"
MDD = "mdd"
MIXED_BIPOLAR = "mixed bipolar"
PURE_MANIA = "pure mania"

In [28]:
from scipy.stats import multinomial
from typing import List
import random

def markov_sequence(p_init: np.array, p_transition: np.array, sequence_length: int) -> List[int]:
    """
    Generate a Markov sequence based on p_init and p_transition.
    """
    initial_state = 0
    states = [initial_state]
    for _ in range(sequence_length - 1):
        p_tr = p_transition[states[-1]]
        new_state = random.choices(range(len(p_tr)), weights=p_tr)[0]
        states.append(new_state)
    return states

In [29]:
p_init = np.array([1.0, 0.0, 0.0])
p_transition = np.array(
    [[0.1, 0.2, 0.3],
     [0.5, 0.3, 0.2],
     [0.3, 0.3, 0.4]]
)

In [30]:
state_sequence = markov_sequence(p_init, p_transition, sequence_length=52)

def eval_sequence(state_sequence):
    """
    Evaluate a state sequence and return the diagnosis.
    """
    MANIC = 1
    DEPRESSED = 2
    arr = np.array(state_sequence)
    cont_depressed = False
    manic_trans = False

    curr = 0
    for i in range(len(arr)):
        if arr[i] == DEPRESSED:
            curr += 1
        elif arr[i] == MANIC:
            manic_trans = True
            curr = 0
        else:
            curr = 0
        if curr >= 2:
            cont_depressed = True
    return cont_depressed, manic_trans

def diagnosis(cont_depressed, manic_trans):
    if cont_depressed and manic_trans:
        return MIXED_BIPOLAR
    elif cont_depressed:
        return MDD
    elif manic_trans:
        return PURE_MANIA
    else:
        return NORMAL




In [31]:
state_sequence

[0,
 1,
 1,
 0,
 0,
 2,
 2,
 1,
 0,
 2,
 0,
 2,
 2,
 2,
 1,
 0,
 2,
 0,
 0,
 2,
 2,
 0,
 0,
 2,
 0,
 2,
 0,
 2,
 2,
 1,
 0,
 2,
 1,
 0,
 2,
 1,
 0,
 2,
 1,
 2,
 2,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 2,
 1,
 0]

In [32]:
def simulate_population(p_init, p_transition, sequence_length=52, population_size = 1000):
    population = {NORMAL : 0, MDD : 0, PURE_MANIA: 0, MIXED_BIPOLAR : 0}
    for sim in range(population_size):
        state_sequence = markov_sequence(p_init, p_transition, sequence_length)
        diag = diagnosis(*eval_sequence(state_sequence))
        population[diag] += 1
    return population


def output_probabilities(population):
    population_size = sum(population.values())
    for k, v in population.items():
        print(f"{k}: {v/population_size}")

output_probabilities(simulate_population(p_init, p_transition))

normal: 0.0
mdd: 0.0
pure mania: 0.002
mixed bipolar: 0.998


# DEAP

In [47]:
import random
from deap import base, creator, tools, algorithms

intended_mdd_percentage = 0.10
intended_pure_mania_mixed_bipolar_percentage = 0.01 

# Define the DEAP types and fitness
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

# Define the evolutionary algorithm parameters
POPULATION_SIZE = 200
generations = 100
mutation_probability = 0.2
crossover_probability = 0.8

# Initialize DEAP toolbox
toolbox = base.Toolbox()

def generate():
    arr = [random.uniform(0, 1) for _ in range(9)]
    # Normalize the rows
    # for i in range(0, 9, 3):
    #     s = sum(arr[i:i+3])
    #     if s == 0:
    #         arr[i:i+3] = [1/3, 1/3, 1/3]
    #         continue
    #     for j in range(3):
    #         arr[i+j] /= s
    # # print(arr)
    return arr

# Define the transition matrix representation (3x3)
matrix_size = 3  # Number of states
toolbox.register("individual", tools.initIterate, creator.Individual, generate)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Define the genetic operators
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=mutation_probability)
toolbox.register("select", tools.selTournament, tournsize=3)

# Define the evaluation function
def evaluate(individual):
    # Reshape the flat matrix into a 2D matrix
    transition_matrix = [individual[i:i+matrix_size] for i in range(0, len(individual), matrix_size)]
    # print(transition_matrix)

    # normalize the rows
    for i in range(0, 3):
        s = sum(transition_matrix[i])
        if s == 0:
            transition_matrix[i] = [1/3, 1/3, 1/3]
            continue
        for j in range(3):
            transition_matrix[i][j] /= s

    # Simulate the population
    population = simulate_population(p_init, transition_matrix, sequence_length=52, population_size=POPULATION_SIZE)
    
    
    # Calculate the percentage of MDD in the sequence
    mdd_percentage = population[MDD] / POPULATION_SIZE
    
    # Calculate the percentage of Pure Mania + Mixed Bipolar in the sequence
    pure_mania_mixed_bipolar_percentage = (population[PURE_MANIA] + population[MIXED_BIPOLAR]) / POPULATION_SIZE
    
    # Calculate the difference between Pure Mania and Mixed Bipolar
    pure_mania_mixed_bipolar_difference = abs(population[PURE_MANIA] - population[MIXED_BIPOLAR])
    
    # print(f"MDD: {mdd_percentage}, Pure Mania + Mixed Bipolar: {pure_mania_mixed_bipolar_percentage}, Difference: {pure_mania_mixed_bipolar_difference}")

    # Fitness: minimize the difference from the intended percentages
    fitness = abs(mdd_percentage - intended_mdd_percentage) + \
              abs(pure_mania_mixed_bipolar_percentage - intended_pure_mania_mixed_bipolar_percentage) + \
              abs(pure_mania_mixed_bipolar_difference)
    
    return fitness,

toolbox.register("evaluate", evaluate)


# Create the population
pop = toolbox.population(n=POPULATION_SIZE)

fitnesses = map(toolbox.evaluate, pop)
for ind, fit in zip(pop, fitnesses):
    ind.fitness.values = fit

# Generate offspring
for __ in range(generations):
    offspring = toolbox.select(pop, len(pop))
    offspring = list(map(toolbox.clone, offspring))  # Generate a deep copy
    
    # Apply crossover
    for c1, c2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < crossover_probability:
            toolbox.mate(c1, c2)
            # Reset their fitness values
            del c1.fitness.values
            del c2.fitness.values

    # Mutate those selected
    for mutant in offspring:
        if random.random() < mutation_probability:
            toolbox.mutate(mutant)
            # Reset fitness values
            del mutant.fitness.values

    # Evaluate non-evaluated individuals in offspring
    invalid_inds = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_inds)
    for ind, fit in zip(invalid_inds, fitnesses):
        ind.fitness.values = fit

    # Replace entire population by the offspring
    pop[:] = offspring



# Extract the best solution
best_solution = tools.selBest(pop, 1)[0]
best_transition_matrix = [best_solution[i:i+matrix_size] for i in range(0, len(best_solution), matrix_size)]

print("Best Transition Matrix:")
for row in best_transition_matrix:
    print(row)

# Simulate the population using the best transition matrix
population = simulate_population(p_init, best_transition_matrix)
output_probabilities(population)

Best Transition Matrix:
[0.49098499624462855, 0.0, 0.0]
[1.0, 0.0, 1.0]
[0.0, 1.0, 1.0]
normal: 1.0
mdd: 0.0
pure mania: 0.0
mixed bipolar: 0.0
